# Creating a model and training it for generating and clustering music

In [1]:
#TODO: Create models: VAE, WAE, U-NET
#Save model weights
#Evaluate in this file?

In [1]:
#libraries
import deeplay as dl
import deeptrack as dt
import torch
import os
from PIL import Image, ImageOps
import torchvision.transforms as T
import matplotlib.pyplot as plt
import numpy as np
from skimage.color import rgb2gray

In [3]:
#Preprocess data
data_dir = os.path.expanduser("./trainImages")

# Load image files using ImageFolder
trainFiles = dt.sources.ImageFolder(root=data_dir)

print(f"Number of train images: {len(trainFiles)}")
#print(f"Number of test images: {len(test_files)}")

Number of train images: 24984


In [6]:
#Create image pipeline
image_pip = (dt.LoadImage(trainFiles.path) >> dt.NormalizeMinMax()
             >> dt.MoveAxis(2, 0) >> dt.pytorch.ToTensor(dtype=torch.float))

In [ ]:
img_tensor = image_pip(trainFiles.path[0])
print(f"The size of each image is: {img_tensor.shape}")

The size of each image is: torch.Size([1, 128, 1907])


In [14]:
#VAE
vae = dl.VariationalAutoEncoder(input_size=(128,1907),
    latent_dim=2, channels=[32, 64],
    reconstruction_loss=torch.nn.BCELoss(reduction="sum"), beta=1,
).create()

#print(vae)

In [ ]:
#Giovanni used image_pip & image_pip here, do not really know why // L-Thor
train_dataset = dt.pytorch.Dataset(image_pip & image_pip, inputs=trainFiles)
train_loader = dl.DataLoader(train_dataset, batch_size=64, shuffle=True)

In [ ]:
#NOTE: Training works, no errors, but it is VERY slow and my computer semi-freezes when I do it, perhaps something is wrong,
#or my computer is trash or maybe we just have to thug it out // L-Thor
vae_trainer = dl.Trainer(max_epochs=2, accelerator="auto")
vae_trainer.fit(vae, train_loader)

In [ ]:
#WAE, let us just focus on the VAE for now, later we can use this code
wae = dl.WassersteinAutoEncoder(
    channels=[32, 64, 128], latent_dim=20,
    reconstruction_loss=torch.nn.MSELoss(reduction="mean"),
).create()

#print(wae)